In [1]:
from names import util
from names import bpe
import pandas as pd
from names import KL_compare
from multiprocessing.pool import Pool
import itertools
from unidecode import unidecode
from string import ascii_lowercase


C:\Users\johan\anaconda3\envs\names\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
data = util.just_give_me_all_the_shit("firstName")
# data

In [3]:
euro_g = ["dutch","english","german","swedish"]
euro_r = ["french","italian","portuguese","spanish"]
# lang_words = util.get_language_corpus_words(language="dutch")



In [4]:
data = []
for language in euro_g:
    lang_words = util.get_language_corpus_words(language=language)
    for word in lang_words:
        data.append(["Euro Germanic", language, word])
for language in euro_r:
    lang_words = util.get_language_corpus_words(language=language)
    for word in lang_words:
        data.append(["Euro Romance", language, word])
df = pd.DataFrame(data, columns = ["family","country","name"])
df['name'] = (
        df['name'].apply(unidecode)
        .str.lower()
        .str.strip()
        .str.replace(r'\s+', '-', regex=True)
        .str.replace(r'[^a-z\-]', '', regex=True)
    )
df = df.groupby(['country', 'name','family']).value_counts().rename("normed_count").reset_index()
df = df[df["normed_count"]>1]
full_df = df.copy()               

In [5]:
family_tokenizers = {family_name: bpe.deserialize_fast_tokenizer(f'{family_name}', None) for family_name in full_df.family.unique()}
data = util.just_give_me_all_the_shit("firstName")

todo = [(name, df, vocab_family_name, tokenizer.get_vocab()) for vocab_family_name, tokenizer in family_tokenizers.items() for name, df in itertools.chain(full_df.groupby('family'), full_df.groupby('country'))]
todo2 = []
names = set(data.name)
for name, df, vocab_family_name, tokenizer in todo:
    vocab = {}
    for key,value in tokenizer.items():
        if key in names or key in list(ascii_lowercase):
            pass
        else:
            vocab[key]=value
    todo2.append((name,df,vocab,tokenizer))
            
            
    
with Pool(processes=6) as pool: # 3 families, longest takes around 
    dists = pd.DataFrame(pool.starmap(KL_compare.get_vocab_counts, todo2))

In [44]:
len(vocab)

456

In [45]:
dists

,group_name,vocab_family_name,counts
0,Euro Germanic,"{'zo': 794, 'domin': 547, 'corin': 1481, 'aike...","{'ulla': 30, 'zo': 3138, 'domin': 28, 'hampus'..."
1,Euro Romance,"{'zo': 794, 'domin': 547, 'corin': 1481, 'aike...","{'ulla': 547, 'zo': 1499, 'domin': 199, 'hampu..."
2,dutch,"{'zo': 794, 'domin': 547, 'corin': 1481, 'aike...","{'ulla': 0, 'zo': 2929, 'domin': 6, 'hampus': ..."
3,english,"{'zo': 794, 'domin': 547, 'corin': 1481, 'aike...","{'ulla': 0, 'zo': 32, 'domin': 11, 'hampus': 0..."
4,french,"{'zo': 794, 'domin': 547, 'corin': 1481, 'aike...","{'ulla': 0, 'zo': 142, 'domin': 9, 'hampus': 0..."
5,german,"{'zo': 794, 'domin': 547, 'corin': 1481, 'aike...","{'ulla': 0, 'zo': 170, 'domin': 0, 'hampus': 0..."
6,italian,"{'zo': 794, 'domin': 547, 'corin': 1481, 'aike...","{'ulla': 547, 'zo': 317, 'domin': 11, 'hampus'..."
7,portuguese,"{'zo': 794, 'domin': 547, 'corin': 1481, 'aike...","{'ulla': 0, 'zo': 390, 'domin': 177, 'hampus':..."
8,spanish,"{'zo': 794, 'domin': 547, 'corin': 1481, 'aike...","{'ulla': 0, 'zo': 650, 'domin': 2, 'hampus': 0..."
9,swedish,"{'zo': 794, 'domin': 547, 'corin': 1481, 'aike...","{'ulla': 30, 'zo': 7, 'domin': 11, 'hampus': 0..."
